In [2]:
#importações necessárias

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


%matplotlib inline

import statsmodels.api as sm
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D

# Para ter melhor print
from IPython.display import display

In [3]:
dados = pd.read_csv("pnad23_filtrado.csv")
dados.head(5)

,Estado,Sexo,Idade,Cor_ou_Raça,Escolaridade,Ocupação,Faixa_de_rendimento,Rendimento_Bruto
0,Rondônia,Homem,30,Parda,Regular do ensino médio ou do 2º grau,Conta Própria,[3SM]+1 a [5SM],5000.0
1,Rondônia,Mulher,50,Parda,Superior - graduação,Empregado do setor público (inclusive empresas...,[2SM]+1 a [3SM],3500.0
2,Rondônia,Mulher,54,Branca,Superior - graduação,Empregado do setor público (inclusive empresas...,[2SM]+1 a [3SM],3700.0
3,Rondônia,Homem,36,Parda,Regular do ensino fundamental ou do 1º grau,Conta Própria,[2SM]+1 a [3SM],3300.0
4,Rondônia,Homem,33,Branca,Regular do ensino médio ou do 2º grau,Conta Própria,[2SM]+1 a [3SM],3500.0


## O projeto
Com o objetivo de prever o salário mais provável de uma pessoa, baseando-se nas suas características socioeconômicas, este projeto utiliza os microdados da PNAD Contínua.
Para construir um modelo preditivo, foi feito a escolha da nossa Target, sendo esta o rendimento bruto, e a seleção das variáveis explicativas , sendo estas escolaridade, idade, gênero, localização geográfica, ocupação, cor/raça e faixa de rendimento. 
A análise exploratória dos dados foi feita no arquivo Jupyter [analise_exploratória.iypnb](analise_exploratoria.ipynb)

Para atingir o objetivo foi necessário aplicar dois modelos preditivos, cujas precisões e resultados foram comparados a fim de decidir qual o modelo mais adequado para o porblema.




## Separando a base de treino e teste 

In [4]:
dados.isnull().sum()

Estado                 0
Sexo                   0
Idade                  0
Cor_ou_Raça            0
Escolaridade           0
Ocupação               0
Faixa_de_rendimento    0
Rendimento_Bruto       0
dtype: int64

In [5]:
dados.dtypes

Estado                  object
Sexo                    object
Idade                    int64
Cor_ou_Raça             object
Escolaridade            object
Ocupação                object
Faixa_de_rendimento     object
Rendimento_Bruto       float64
dtype: object

In [6]:
x_colunas = dados[['Estado', 'Sexo', 'Idade', 'Cor_ou_Raça', 'Escolaridade', 'Ocupação','Faixa_de_rendimento']]

y_colunas = dados[['Rendimento_Bruto']]


### Para separar as bases em treino e teste é necessário aplicar o get_dummies(), para as variáveis categoricas poderem ser utilizadas no modelo

In [7]:
x_dummies = pd.get_dummies(x_colunas)
x_dummies = x_dummies.astype(int)
x_dummies.dtypes

Idade                                   int32
Estado_Acre                             int32
Estado_Alagoas                          int32
Estado_Amapá                            int32
Estado_Amazonas                         int32
                                        ...  
Faixa_de_rendimento_[1SM]+1 a [2SM]     int32
Faixa_de_rendimento_[20SM]+1 ou mais    int32
Faixa_de_rendimento_[2SM]+1 a [3SM]     int32
Faixa_de_rendimento_[3SM]+1 a [5SM]     int32
Faixa_de_rendimento_[5SM]+1 a [10SM]    int32
Length: 65, dtype: object

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_dummies ,y_colunas , test_size=0.33, random_state= 42 )

In [15]:
x_train

,Idade,Estado_Acre,Estado_Alagoas,Estado_Amapá,Estado_Amazonas,Estado_Bahia,Estado_Ceará,Estado_Distrito Federal,Estado_Espirito Santo,Estado_Goiás,...,"Ocupação_Militar do exército, da marinha, da aeronáutica, da polícia militar ou do corpo de bombeiros militar",Ocupação_Trabalhador doméstico,"Faixa_de_rendimento_1 a [0,5SM]","Faixa_de_rendimento_[0,5SM]+1 a [1SM]",Faixa_de_rendimento_[10SM]+1 a [20SM],Faixa_de_rendimento_[1SM]+1 a [2SM],Faixa_de_rendimento_[20SM]+1 ou mais,Faixa_de_rendimento_[2SM]+1 a [3SM],Faixa_de_rendimento_[3SM]+1 a [5SM],Faixa_de_rendimento_[5SM]+1 a [10SM]
9255,20,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
83381,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103097,48,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
139202,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
103004,53,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
103694,42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
131932,32,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
146867,51,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### Aplicando o modelo Random Forest

In [10]:
random_model = RandomForestRegressor(n_estimators=100, random_state = 42 )

### Ensinando o modelo, para que consiga encontrar os padrões

In [11]:
random_model.fit(x_train, y_train)

c:\Users\rafav\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


KeyboardInterrupt: 

### Nas celulas abaixo utilizamos as métricas MAE(Erro médio absoluto), MSE(Erro médio quadrático) e $R^2$(Coeficiente de determinação que mede quanto da variância dos modelos reais é explicada pelo modelo)

In [ ]:
y_prev_train = random_model.predict(x_train)
mae = mean_absolute_error(y_train, y_prev_train)
mse = mean_squared_error(y_train, y_prev_train)
r2 = r2_score(y_train, y_prev_train)
mae, mse, r2

(210.69812439007617, 463390.5781934575, 0.971475393335519)

O MAE indica que o modelo errou, em média, 210.69 unidades nas previsões. Um erro baixo, indicando uma boa performance nos dados de treino.

O valor do MSE é 46390.57. Esse número é maior devido à penalização de erros maiores.

O valor de $R^2$ foi de 0.97, indicando que o modelo está ajustado para a base de treino, mas o bom desempenho não garante que seja bom com os dados de teste.

In [ ]:
y_prev = random_model.predict(x_test)
mse = mean_squared_error(y_test, y_prev)
r2 = r2_score(y_test, y_prev)
mae = mean_absolute_error(y_test, y_prev)
mae, mse, r2 

(434.50358136448807, 3465863.87181802, 0.7714871476759115)

O MAE indica que o modelo errou, em média, 434.50 unidades nas previsões. Valor maior do que nos dados de treino. Indicando dificuldade em generalizar.

O valor do MSE é 3465863.87. Mostrando um aumento considerável nos erros grandes.

O valor de $R^2$ foi de 0.77, mostrando que ainda explica boa parte da variância nos dados de teste, mas com desempenho inferior do que com os dados de treino.

O modelo se saiu muito bem com a base de treino, porém seu desempenho caiu quando foi  utilizada a base de teste, sugerindo um pouco de overfitting.
Overfitting é justamente quando um modelo se sai muito bem com os dados de treino, porém seu desempenho cai com os dados de treino.

___
<div id='regressao'></div>

## Modelo de regressão


Nessa etapa será aplicado à base de dados o mesmo modelo de regressão linear visto na <strong>AULA 28</strong>

In [19]:
# Função utilitária para fazer a regressão 
# com constante adicionada

def regress(Y,X):
    '''
    Y: coluna do DataFrame utilizada como variável resposta (TARGET)
    X: coluna(s) do DataFrame utilizadas como variável(is) explicativas (FEATURES)
    '''
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    
    return results

A seguir serão removidas as colunas da base de dados `x_train` que possuirem valor-p menor do que o nível de significância ($\alpha$) estabelecido pelo grupo como 5%

In [20]:
regressao = regress(y_train, x_train)
regressao.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     8389.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:19   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117311   BIC:                         2.089e+06
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4807.0563     60.660     79.247      0.000    4688.165    4925.948
Idade                                                                                                             1.2426      0.469      2.650      0.008       0.323       2.162
Estado_Acre                                                                                                     221.7339     41.000      5.408      0.000     141.375     302.093
Estado_Alagoas                                                                                                  161.2207     28.457      5.665      0.000     105.446     216.995
Estado_Amapá                                                                                                    240.6047     52.879      4.550      0.000     136.964     344.246
Estado_Amazonas                                                                                                 224.6591     31.548      7.121      0.000     162.826     286.493
Estado_Bahia                                                                                                    153.2930     25.625      5.982      0.000     103.069     203.517
Estado_Ceará                                                                                                    146.0079     25.990      5.618      0.000      95.069     196.947
Estado_Distrito Federal                                                                                         195.7662     34.449      5.683      0.000     128.247     263.285
Estado_Espirito Santo                                                                                           165.6646     25.809      6.419      0.000     115.079     216.250
Estado_Goiás                                                                                                    209.7985     26.055      8.052      0.000     158.731     260.866
Estado_Maranhão                                                                                                 151.0464     23.846      6.334      0.000     104.308     197.785
Estado_Mato Grosso                                                                                              198.9612     29.650      6.710      0.000     140.848     257.074
Estado_Mato Grosso do Sul                                                                                       257.3822     31.626      8.138      0.000     195.395     319.369
Estado_Minas Gerais                                      

A seguir serão criados novos dataframes nos quais, a cada uso da função `regress()` será removida a coluna que possuir maior valor na coluna `P>|t|`.<br>
Nesse caso, será a coluna: `Escolaridade_Creche (disponível apenas no questionário anual de educação)`

In [21]:
x_train2 = x_train.drop(columns=["Escolaridade_Creche (disponível apenas no questionário anual de educação)"])

O processo será repetido diversas vezes até que todos os valores da coluna sejam menores do que 5%

In [22]:
regressao2 = regress(y_train, x_train2)
regressao2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     8389.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:20   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117311   BIC:                         2.089e+06
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4975.1208    445.698     11.163      0.000    4101.560    5848.682
Idade                                                                                                             1.2426      0.469      2.650      0.008       0.323       2.162
Estado_Acre                                                                                                     227.9585     43.980      5.183      0.000     141.758     314.159
Estado_Alagoas                                                                                                  167.4453     32.848      5.098      0.000     103.064     231.826
Estado_Amapá                                                                                                    246.8293     55.361      4.459      0.000     138.322     355.336
Estado_Amazonas                                                                                                 230.8837     35.557      6.493      0.000     161.192     300.576
Estado_Bahia                                                                                                    159.5176     30.438      5.241      0.000      99.859     219.177
Estado_Ceará                                                                                                    152.2325     30.649      4.967      0.000      92.161     212.304
Estado_Distrito Federal                                                                                         201.9908     38.145      5.295      0.000     127.227     276.755
Estado_Espirito Santo                                                                                           171.8892     30.471      5.641      0.000     112.166     231.612
Estado_Goiás                                                                                                    216.0231     30.775      7.019      0.000     155.704     276.342
Estado_Maranhão                                                                                                 157.2710     28.871      5.447      0.000     100.684     213.858
Estado_Mato Grosso                                                                                              205.1858     33.867      6.059      0.000     138.808     271.564
Estado_Mato Grosso do Sul                                                                                       263.6068     35.611      7.402      0.000     193.809     333.404
Estado_Minas Gerais                                      

In [23]:
x_train3 = x_train2.drop(columns=['Escolaridade_Superior - graduação'])
regressao3 = regress(y_train, x_train3)
regressao3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     8533.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:21   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117312   BIC:                         2.089e+06
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4991.5914     54.472     91.636      0.000    4884.827    5098.356
Idade                                                                                                             1.2425      0.469      2.649      0.008       0.323       2.162
Estado_Acre                                                                                                     228.5515     40.995      5.575      0.000     148.202     308.901
Estado_Alagoas                                                                                                  168.0573     28.438      5.910      0.000     112.318     223.796
Estado_Amapá                                                                                                    247.4420     52.859      4.681      0.000     143.838     351.046
Estado_Amazonas                                                                                                 231.4962     31.524      7.344      0.000     169.710     293.282
Estado_Bahia                                                                                                    160.1303     25.607      6.253      0.000     109.942     210.319
Estado_Ceará                                                                                                    152.8382     25.975      5.884      0.000     101.927     203.749
Estado_Distrito Federal                                                                                         202.6021     34.431      5.884      0.000     135.118     270.087
Estado_Espirito Santo                                                                                           172.4928     25.801      6.686      0.000     121.923     223.062
Estado_Goiás                                                                                                    216.6338     26.039      8.320      0.000     165.597     267.670
Estado_Maranhão                                                                                                 157.8779     23.829      6.625      0.000     111.173     204.583
Estado_Mato Grosso                                                                                              205.7962     29.636      6.944      0.000     147.711     263.881
Estado_Mato Grosso do Sul                                                                                       264.2172     31.613      8.358      0.000     202.256     326.178
Estado_Minas Gerais                                      

In [24]:
x_train4 = x_train3.drop(columns=['Escolaridade_Antigo científico, clássico, etc. (médio 2º ciclo)'])
regressao4 = regress(y_train, x_train4)
regressao4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     8683.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:22   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117313   BIC:                         2.089e+06
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4991.4857     54.469     91.640      0.000    4884.728    5098.243
Idade                                                                                                             1.2543      0.464      2.702      0.007       0.345       2.164
Estado_Acre                                                                                                     228.5358     40.995      5.575      0.000     148.187     308.884
Estado_Alagoas                                                                                                  168.0793     28.438      5.910      0.000     112.341     223.817
Estado_Amapá                                                                                                    247.3501     52.857      4.680      0.000     143.752     350.948
Estado_Amazonas                                                                                                 231.4609     31.523      7.343      0.000     169.676     293.245
Estado_Bahia                                                                                                    160.1229     25.607      6.253      0.000     109.934     210.311
Estado_Ceará                                                                                                    152.8351     25.975      5.884      0.000     101.924     203.746
Estado_Distrito Federal                                                                                         202.6480     34.430      5.886      0.000     135.166     270.130
Estado_Espirito Santo                                                                                           172.5294     25.800      6.687      0.000     121.962     223.097
Estado_Goiás                                                                                                    216.6907     26.037      8.322      0.000     165.658     267.723
Estado_Maranhão                                                                                                 157.8419     23.828      6.624      0.000     111.139     204.545
Estado_Mato Grosso                                                                                              205.7665     29.635      6.943      0.000     147.682     263.851
Estado_Mato Grosso do Sul                                                                                       264.2426     31.612      8.359      0.000     202.283     326.203
Estado_Minas Gerais                                      

In [25]:
x_train5 = x_train4.drop(columns=['Escolaridade_Pré-escola'])
regressao5 = regress(y_train, x_train5)
regressao5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     8838.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:22   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117314   BIC:                         2.089e+06
Df Model:                          56                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4991.4146     54.468     91.639      0.000    4884.657    5098.172
Idade                                                                                                             1.2345      0.464      2.663      0.008       0.326       2.143
Estado_Acre                                                                                                     228.6609     40.994      5.578      0.000     148.313     309.009
Estado_Alagoas                                                                                                  168.1846     28.438      5.914      0.000     112.447     223.922
Estado_Amapá                                                                                                    247.4498     52.856      4.682      0.000     143.852     351.048
Estado_Amazonas                                                                                                 231.5924     31.523      7.347      0.000     169.809     293.376
Estado_Bahia                                                                                                    159.8057     25.604      6.241      0.000     109.622     209.989
Estado_Ceará                                                                                                    152.9491     25.975      5.888      0.000     102.039     203.859
Estado_Distrito Federal                                                                                         202.7131     34.430      5.888      0.000     135.231     270.195
Estado_Espirito Santo                                                                                           172.6250     25.800      6.691      0.000     122.058     223.192
Estado_Goiás                                                                                                    216.5298     26.036      8.316      0.000     165.499     267.561
Estado_Maranhão                                                                                                 157.7912     23.828      6.622      0.000     111.088     204.494
Estado_Mato Grosso                                                                                              205.4641     29.633      6.934      0.000     147.384     263.544
Estado_Mato Grosso do Sul                                                                                       264.2422     31.612      8.359      0.000     202.282     326.202
Estado_Minas Gerais                                      

In [26]:
x_train6 = x_train5.drop(columns=['Escolaridade_Mestrado'])
regressao6 = regress(y_train, x_train6)
regressao6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     8999.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:23   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117315   BIC:                         2.089e+06
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4993.3221     54.449     91.706      0.000    4886.602    5100.042
Idade                                                                                                             1.2377      0.464      2.669      0.008       0.329       2.146
Estado_Acre                                                                                                     228.3624     40.994      5.571      0.000     148.015     308.710
Estado_Alagoas                                                                                                  168.4507     28.437      5.924      0.000     112.714     224.187
Estado_Amapá                                                                                                    247.1351     52.856      4.676      0.000     143.538     350.732
Estado_Amazonas                                                                                                 231.8520     31.522      7.355      0.000     170.069     293.635
Estado_Bahia                                                                                                    160.1856     25.603      6.257      0.000     110.005     210.366
Estado_Ceará                                                                                                    153.3313     25.973      5.903      0.000     102.424     204.239
Estado_Distrito Federal                                                                                         203.5570     34.424      5.913      0.000     136.086     271.028
Estado_Espirito Santo                                                                                           172.9968     25.798      6.706      0.000     122.432     223.561
Estado_Goiás                                                                                                    216.3287     26.036      8.309      0.000     165.298     267.359
Estado_Maranhão                                                                                                 157.8262     23.828      6.623      0.000     111.123     204.530
Estado_Mato Grosso                                                                                              205.2706     29.633      6.927      0.000     147.191     263.350
Estado_Mato Grosso do Sul                                                                                       264.0459     31.612      8.353      0.000     202.087     326.005
Estado_Minas Gerais                                      

In [27]:
x_train7 = x_train6.drop(columns=['Cor_ou_Raça_Ignorado'])
regressao7 = regress(y_train, x_train7)
regressao7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     8999.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:24   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117315   BIC:                         2.089e+06
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5418.1345    341.650     15.859      0.000    4748.505    6087.764
Idade                                                                                                             1.2377      0.464      2.669      0.008       0.329       2.146
Estado_Acre                                                                                                     244.0962     42.869      5.694      0.000     160.073     328.119
Estado_Alagoas                                                                                                  184.1845     31.072      5.928      0.000     123.284     245.084
Estado_Amapá                                                                                                    262.8689     54.326      4.839      0.000     156.390     369.348
Estado_Amazonas                                                                                                 247.5858     33.937      7.295      0.000     181.070     314.102
Estado_Bahia                                                                                                    175.9194     28.504      6.172      0.000     120.052     231.786
Estado_Ceará                                                                                                    169.0651     28.834      5.863      0.000     112.551     225.579
Estado_Distrito Federal                                                                                         219.2907     36.432      6.019      0.000     147.884     290.698
Estado_Espirito Santo                                                                                           188.7306     28.586      6.602      0.000     132.703     244.759
Estado_Goiás                                                                                                    232.0625     28.899      8.030      0.000     175.421     288.704
Estado_Maranhão                                                                                                 173.5600     26.873      6.458      0.000     120.889     226.231
Estado_Mato Grosso                                                                                              221.0044     32.179      6.868      0.000     157.934     284.074
Estado_Mato Grosso do Sul                                                                                       279.7797     34.014      8.225      0.000     213.113     346.446
Estado_Minas Gerais                                      

In [28]:
x_train8 = x_train7.drop(columns=['Cor_ou_Raça_Amarela'])
regressao8 = regress(y_train, x_train8)
regressao8.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     9165.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:25   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117316   BIC:                         2.089e+06
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5400.7010     37.002    145.958      0.000    5328.178    5473.224
Idade                                                                                                             1.2377      0.464      2.670      0.008       0.329       2.146
Estado_Acre                                                                                                     243.4459     40.954      5.944      0.000     163.176     323.716
Estado_Alagoas                                                                                                  183.5366     28.392      6.464      0.000     127.888     239.185
Estado_Amapá                                                                                                    262.2196     52.833      4.963      0.000     158.667     365.772
Estado_Amazonas                                                                                                 246.9380     31.503      7.838      0.000     185.192     308.684
Estado_Bahia                                                                                                    175.2719     25.560      6.857      0.000     125.174     225.370
Estado_Ceará                                                                                                    168.4182     25.934      6.494      0.000     117.589     219.248
Estado_Distrito Federal                                                                                         218.6780     34.421      6.353      0.000     151.214     286.142
Estado_Espirito Santo                                                                                           188.0965     25.778      7.297      0.000     137.571     238.622
Estado_Goiás                                                                                                    231.4135     25.985      8.906      0.000     180.483     282.344
Estado_Maranhão                                                                                                 172.9174     23.779      7.272      0.000     126.310     219.524
Estado_Mato Grosso                                                                                              220.3554     29.590      7.447      0.000     162.360     278.351
Estado_Mato Grosso do Sul                                                                                       279.1285     31.559      8.845      0.000     217.273     340.984
Estado_Minas Gerais                                      

In [29]:
x_train9 = x_train8.drop(columns=['Cor_ou_Raça_Indigena'])
regressao9 = regress(y_train, x_train9)
regressao9.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     9338.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:26   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117317   BIC:                         2.089e+06
Df Model:                          53                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5417.0692     29.912    181.097      0.000    5358.441    5475.697
Idade                                                                                                             1.2364      0.464      2.667      0.008       0.328       2.145
Estado_Acre                                                                                                     243.8527     40.951      5.955      0.000     163.590     324.115
Estado_Alagoas                                                                                                  184.0742     28.383      6.485      0.000     128.444     239.705
Estado_Amapá                                                                                                    262.7496     52.828      4.974      0.000     159.207     366.292
Estado_Amazonas                                                                                                 248.7770     31.408      7.921      0.000     187.218     310.336
Estado_Bahia                                                                                                    175.8651     25.548      6.884      0.000     125.791     225.939
Estado_Ceará                                                                                                    169.0966     25.918      6.524      0.000     118.298     219.895
Estado_Distrito Federal                                                                                         219.1920     34.414      6.369      0.000     151.741     286.643
Estado_Espirito Santo                                                                                           188.7322     25.765      7.325      0.000     138.234     239.230
Estado_Goiás                                                                                                    231.8793     25.978      8.926      0.000     180.964     282.795
Estado_Maranhão                                                                                                 173.3194     23.773      7.291      0.000     126.724     219.915
Estado_Mato Grosso                                                                                              220.8591     29.582      7.466      0.000     162.878     278.840
Estado_Mato Grosso do Sul                                                                                       279.6187     31.552      8.862      0.000     217.777     341.461
Estado_Minas Gerais                                      

In [30]:
x_train10 = x_train9.drop(columns=['Escolaridade_Alfabetização de jovens e adultos'])
regressao9 = regress(y_train, x_train10)
regressao9.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     9518.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:27   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117318   BIC:                         2.089e+06
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5416.8709     29.912    181.091      0.000    5358.243    5475.499
Idade                                                                                                             1.1925      0.463      2.577      0.010       0.286       2.099
Estado_Acre                                                                                                     243.7344     40.951      5.952      0.000     163.472     323.997
Estado_Alagoas                                                                                                  184.0403     28.383      6.484      0.000     128.410     239.671
Estado_Amapá                                                                                                    263.4539     52.827      4.987      0.000     159.915     366.993
Estado_Amazonas                                                                                                 249.1572     31.407      7.933      0.000     187.599     310.715
Estado_Bahia                                                                                                    175.6123     25.548      6.874      0.000     125.539     225.686
Estado_Ceará                                                                                                    169.0555     25.918      6.523      0.000     118.257     219.854
Estado_Distrito Federal                                                                                         219.4603     34.414      6.377      0.000     152.010     286.911
Estado_Espirito Santo                                                                                           188.8019     25.765      7.328      0.000     138.304     239.300
Estado_Goiás                                                                                                    232.0734     25.977      8.934      0.000     181.158     282.989
Estado_Maranhão                                                                                                 173.2722     23.773      7.289      0.000     126.677     219.868
Estado_Mato Grosso                                                                                              220.6786     29.582      7.460      0.000     162.698     278.659
Estado_Mato Grosso do Sul                                                                                       279.6057     31.552      8.862      0.000     217.763     341.448
Estado_Minas Gerais                                      

In [31]:
x_train11 = x_train10.drop(columns=['Cor_ou_Raça_Parda'])
regressao10 = regress(y_train, x_train11)
regressao10.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     9704.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:27   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117319   BIC:                         2.089e+06
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5463.8472     15.196    359.567      0.000    5434.064    5493.630
Idade                                                                                                             1.1818      0.463      2.554      0.011       0.275       2.089
Estado_Acre                                                                                                     245.4136     40.941      5.994      0.000     165.170     325.657
Estado_Alagoas                                                                                                  185.9626     28.364      6.556      0.000     130.370     241.555
Estado_Amapá                                                                                                    265.6601     52.813      5.030      0.000     162.147     369.173
Estado_Amazonas                                                                                                 249.4182     31.407      7.941      0.000     187.861     310.976
Estado_Bahia                                                                                                    177.4579     25.528      6.952      0.000     127.424     227.492
Estado_Ceará                                                                                                    171.2651     25.890      6.615      0.000     120.521     222.009
Estado_Distrito Federal                                                                                         221.5255     34.395      6.441      0.000     154.111     288.940
Estado_Espirito Santo                                                                                           190.9030     25.739      7.417      0.000     140.455     241.351
Estado_Goiás                                                                                                    234.0611     25.955      9.018      0.000     183.190     284.932
Estado_Maranhão                                                                                                 175.3326     23.747      7.383      0.000     128.789     221.876
Estado_Mato Grosso                                                                                              222.6974     29.562      7.533      0.000     164.757     280.638
Estado_Mato Grosso do Sul                                                                                       280.4890     31.549      8.891      0.000     218.653     342.325
Estado_Minas Gerais                                      

In [32]:
x_train12 = x_train11.drop(columns=['Cor_ou_Raça_Preta'])
regressao12 = regress(y_train, x_train12)
regressao12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     9898.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:28   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117320   BIC:                         2.089e+06
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5465.7180     15.122    361.440      0.000    5436.079    5495.357
Idade                                                                                                             1.1898      0.463      2.572      0.010       0.283       2.097
Estado_Acre                                                                                                     244.2887     40.931      5.968      0.000     164.065     324.513
Estado_Alagoas                                                                                                  185.3557     28.360      6.536      0.000     129.771     240.941
Estado_Amapá                                                                                                    265.4202     52.813      5.026      0.000     161.908     368.933
Estado_Amazonas                                                                                                 247.4172     31.367      7.888      0.000     185.939     308.895
Estado_Bahia                                                                                                    179.8237     25.458      7.064      0.000     129.926     229.721
Estado_Ceará                                                                                                    170.2628     25.878      6.580      0.000     119.543     220.983
Estado_Distrito Federal                                                                                         222.4260     34.388      6.468      0.000     155.026     289.826
Estado_Espirito Santo                                                                                           191.6611     25.732      7.448      0.000     141.227     242.095
Estado_Goiás                                                                                                    233.9403     25.955      9.013      0.000     183.070     284.811
Estado_Maranhão                                                                                                 175.6466     23.745      7.397      0.000     129.106     222.187
Estado_Mato Grosso                                                                                              222.9778     29.561      7.543      0.000     165.039     280.917
Estado_Mato Grosso do Sul                                                                                       280.1673     31.548      8.881      0.000     218.333     342.001
Estado_Minas Gerais                                      

### Removendo a coluna de Raça

Após o processo de remoção das colunas, percebe-se que a coluna `Cor_ou_Raça` possui apenas um único valor (no caso, aqueles de raça branca). Tendo isso em mente, o grupo optou por remover a coluna `Cor_ou_Raça` por completo da base de dados.

In [33]:
x_train13 = x_train12.drop(columns=['Cor_ou_Raça_Branca'])
regressao12 = regress(y_train, x_train13)
regressao12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                 1.010e+04
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:10:29   Log-Likelihood:            -1.0440e+06
No. Observations:              117371   AIC:                         2.088e+06
Df Residuals:                  117321   BIC:                         2.089e+06
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5474.8072     14.913    367.106      0.000    5445.577    5504.037
Idade                                                                                                             1.2746      0.462      2.758      0.006       0.369       2.180
Estado_Acre                                                                                                     237.8496     40.895      5.816      0.000     157.697     318.002
Estado_Alagoas                                                                                                  183.3072     28.356      6.465      0.000     127.730     238.884
Estado_Amapá                                                                                                    257.0930     52.766      4.872      0.000     153.673     360.513
Estado_Amazonas                                                                                                 240.5786     31.311      7.683      0.000     179.209     301.949
Estado_Bahia                                                                                                    174.6336     25.419      6.870      0.000     124.813     224.455
Estado_Ceará                                                                                                    168.3028     25.873      6.505      0.000     117.591     219.014
Estado_Distrito Federal                                                                                         221.5186     34.389      6.442      0.000     154.117     288.920
Estado_Espirito Santo                                                                                           193.4828     25.728      7.520      0.000     143.055     243.910
Estado_Goiás                                                                                                    234.5036     25.956      9.035      0.000     183.631     285.376
Estado_Maranhão                                                                                                 170.2860     23.701      7.185      0.000     123.833     216.739
Estado_Mato Grosso                                                                                              221.5997     29.560      7.497      0.000     163.663     279.537
Estado_Mato Grosso do Sul                                                                                       282.5969     31.543      8.959      0.000     220.774     344.420
Estado_Minas Gerais                                      

### Atualizando a base de dados de teste

Após a remoção das colunas da base de dados de treino, será feito o mesmo processo, porém com a base de dados de teste

In [39]:
regressao_teste = regress(y_test, x_test)
regressao_teste.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     5488.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:20:40   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57750   BIC:                         1.011e+06
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4609.7279     70.872     65.043      0.000    4470.818    4748.637
Idade                                                                                                             3.3593      0.574      5.852      0.000       2.234       4.485
Estado_Acre                                                                                                     211.3621     51.073      4.138      0.000     111.258     311.466
Estado_Alagoas                                                                                                  174.4072     35.418      4.924      0.000     104.989     243.826
Estado_Amapá                                                                                                    233.8565     67.151      3.483      0.000     102.241     365.472
Estado_Amazonas                                                                                                 187.7289     39.261      4.782      0.000     110.778     264.680
Estado_Bahia                                                                                                    152.9243     31.490      4.856      0.000      91.204     214.644
Estado_Ceará                                                                                                    117.8755     31.673      3.722      0.000      55.797     179.954
Estado_Distrito Federal                                                                                          69.6706     42.022      1.658      0.097     -12.693     152.034
Estado_Espirito Santo                                                                                           179.8238     32.302      5.567      0.000     116.511     243.136
Estado_Goiás                                                                                                    324.7178     32.257     10.067      0.000     261.494     387.941
Estado_Maranhão                                                                                                 137.6246     28.822      4.775      0.000      81.133     194.117
Estado_Mato Grosso                                                                                              223.2699     35.787      6.239      0.000     153.127     293.413
Estado_Mato Grosso do Sul                                                                                       171.3524     38.988      4.395      0.000      94.935     247.770
Estado_Minas Gerais                                      

In [40]:
x_test2 = x_test.drop(columns=['Cor_ou_Raça_Ignorado'])
regressao_teste2 = regress(y_test, x_test2)
regressao_teste2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     5488.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:22:27   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57750   BIC:                         1.011e+06
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4678.2215    307.850     15.196      0.000    4074.835    5281.608
Idade                                                                                                             3.3593      0.574      5.852      0.000       2.234       4.485
Estado_Acre                                                                                                     213.8989     52.282      4.091      0.000     111.426     316.372
Estado_Alagoas                                                                                                  176.9440     37.120      4.767      0.000     104.188     249.700
Estado_Amapá                                                                                                    236.3933     68.073      3.473      0.001     102.970     369.817
Estado_Amazonas                                                                                                 190.2657     40.820      4.661      0.000     110.258     270.273
Estado_Bahia                                                                                                    155.4611     33.398      4.655      0.000      90.001     220.922
Estado_Ceará                                                                                                    120.4123     33.499      3.594      0.000      54.754     186.071
Estado_Distrito Federal                                                                                          72.2074     43.497      1.660      0.097     -13.047     157.462
Estado_Espirito Santo                                                                                           182.3606     34.094      5.349      0.000     115.537     249.184
Estado_Goiás                                                                                                    327.2546     34.126      9.590      0.000     260.367     394.142
Estado_Maranhão                                                                                                 140.1614     30.894      4.537      0.000      79.609     200.713
Estado_Mato Grosso                                                                                              225.8067     37.486      6.024      0.000     152.334     299.279
Estado_Mato Grosso do Sul                                                                                       173.8892     40.555      4.288      0.000      94.401     253.377
Estado_Minas Gerais                                      

In [42]:
x_test3 = x_test2.drop(columns=['Escolaridade_Creche (disponível apenas no questionário anual de educação)'])
regressao_teste3 = regress(y_test, x_test3)
regressao_teste3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     5488.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:23:36   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57750   BIC:                         1.011e+06
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4930.2552    886.738      5.560      0.000    3192.245    6668.265
Idade                                                                                                             3.3593      0.574      5.852      0.000       2.234       4.485
Estado_Acre                                                                                                     223.2335     60.699      3.678      0.000     104.263     342.204
Estado_Alagoas                                                                                                  186.2786     48.252      3.861      0.000      91.705     280.852
Estado_Amapá                                                                                                    245.7279     74.730      3.288      0.001      99.257     392.199
Estado_Amazonas                                                                                                 199.6003     50.571      3.947      0.000     100.480     298.720
Estado_Bahia                                                                                                    164.7957     45.445      3.626      0.000      75.724     253.867
Estado_Ceará                                                                                                    129.7469     45.519      2.850      0.004      40.529     218.965
Estado_Distrito Federal                                                                                          81.5420     53.327      1.529      0.126     -22.979     186.063
Estado_Espirito Santo                                                                                           191.6952     45.978      4.169      0.000     101.578     281.812
Estado_Goiás                                                                                                    336.5892     46.006      7.316      0.000     246.416     426.762
Estado_Maranhão                                                                                                 149.4959     43.644      3.425      0.001      63.954     235.038
Estado_Mato Grosso                                                                                              235.1413     48.543      4.844      0.000     139.997     330.286
Estado_Mato Grosso do Sul                                                                                       183.2238     50.953      3.596      0.000      83.356     283.092
Estado_Minas Gerais                                      

In [44]:
x_test4 = x_test3.drop(columns=['Escolaridade_Superior - graduação', 'Escolaridade_Especialização de nível superior'])
regressao_teste4 = regress(y_test, x_test4)
regressao_teste4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     5680.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:26:05   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57752   BIC:                         1.011e+06
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4921.9440    313.942     15.678      0.000    4306.616    5537.272
Idade                                                                                                             3.3792      0.574      5.888      0.000       2.254       4.504
Estado_Acre                                                                                                     221.6736     52.320      4.237      0.000     119.127     324.221
Estado_Alagoas                                                                                                  186.0921     37.190      5.004      0.000     113.200     258.984
Estado_Amapá                                                                                                    243.1472     68.087      3.571      0.000     109.697     376.597
Estado_Amazonas                                                                                                 199.6265     40.910      4.880      0.000     119.443     279.810
Estado_Bahia                                                                                                    164.8564     33.476      4.925      0.000      99.243     230.470
Estado_Ceará                                                                                                    129.5164     33.571      3.858      0.000      63.718     195.315
Estado_Distrito Federal                                                                                          83.1059     43.534      1.909      0.056      -2.222     168.434
Estado_Espirito Santo                                                                                           192.6788     34.159      5.641      0.000     125.726     259.631
Estado_Goiás                                                                                                    336.2310     34.211      9.828      0.000     269.178     403.284
Estado_Maranhão                                                                                                 149.1406     30.982      4.814      0.000      88.416     209.865
Estado_Mato Grosso                                                                                              233.9127     37.553      6.229      0.000     160.309     307.516
Estado_Mato Grosso do Sul                                                                                       183.1573     40.618      4.509      0.000     103.545     262.769
Estado_Minas Gerais                                      

In [45]:
x_test5 = x_test4.drop(columns=['Escolaridade_Mestrado'])
regressao_teste5 = regress(y_test, x_test5)
regressao_teste5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     5782.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:27:07   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57753   BIC:                         1.011e+06
Df Model:                          56                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          4917.7524    313.891     15.667      0.000    4302.525    5532.980
Idade                                                                                                             3.3766      0.574      5.883      0.000       2.252       4.502
Estado_Acre                                                                                                     221.5854     52.319      4.235      0.000     119.039     324.132
Estado_Alagoas                                                                                                  185.9858     37.189      5.001      0.000     113.094     258.877
Estado_Amapá                                                                                                    243.1571     68.086      3.571      0.000     109.707     376.607
Estado_Amazonas                                                                                                 199.4626     40.909      4.876      0.000     119.281     279.644
Estado_Bahia                                                                                                    164.6621     33.475      4.919      0.000      99.051     230.273
Estado_Ceará                                                                                                    129.2899     33.569      3.851      0.000      63.494     195.086
Estado_Distrito Federal                                                                                          82.3839     43.524      1.893      0.058      -2.922     167.690
Estado_Espirito Santo                                                                                           192.3946     34.157      5.633      0.000     125.447     259.343
Estado_Goiás                                                                                                    336.1884     34.211      9.827      0.000     269.136     403.241
Estado_Maranhão                                                                                                 149.0883     30.982      4.812      0.000      88.364     209.813
Estado_Mato Grosso                                                                                              233.9674     37.553      6.230      0.000     160.364     307.571
Estado_Mato Grosso do Sul                                                                                       183.0705     40.618      4.507      0.000     103.459     262.682
Estado_Minas Gerais                                      

In [47]:
x_test6 = x_test5.drop(columns=['Cor_ou_Raça_Amarela'])
regressao_teste6 = regress(y_test, x_test6)
regressao_teste6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     5887.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:28:36   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57754   BIC:                         1.011e+06
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5311.2846     46.460    114.321      0.000    5220.224    5402.345
Idade                                                                                                             3.3734      0.574      5.878      0.000       2.248       4.498
Estado_Acre                                                                                                     236.3815     51.001      4.635      0.000     136.419     336.344
Estado_Alagoas                                                                                                  200.6033     35.357      5.674      0.000     131.304     269.903
Estado_Amapá                                                                                                    257.7852     67.102      3.842      0.000     126.265     389.305
Estado_Amazonas                                                                                                 214.0650     39.254      5.453      0.000     137.127     291.003
Estado_Bahia                                                                                                    179.2193     31.444      5.700      0.000     117.589     240.850
Estado_Ceará                                                                                                    143.5948     31.616      4.542      0.000      81.628     205.562
Estado_Distrito Federal                                                                                          97.2106     41.923      2.319      0.020      15.042     179.380
Estado_Espirito Santo                                                                                           206.6742     32.246      6.409      0.000     143.471     269.877
Estado_Goiás                                                                                                    350.8902     32.185     10.902      0.000     287.807     413.973
Estado_Maranhão                                                                                                 163.7451     28.744      5.697      0.000     107.407     220.083
Estado_Mato Grosso                                                                                              248.6439     35.724      6.960      0.000     178.626     318.662
Estado_Mato Grosso do Sul                                                                                       197.7992     38.921      5.082      0.000     121.514     274.085
Estado_Minas Gerais                                      

In [48]:
x_test7 = x_test6.drop(columns=['Cor_ou_Raça_Preta'])
regressao_teste7 = regress(y_test, x_test7)
regressao_teste7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     5996.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:29:50   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57755   BIC:                         1.011e+06
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5331.9516     20.402    261.347      0.000    5291.964    5371.939
Idade                                                                                                             3.3696      0.574      5.872      0.000       2.245       4.494
Estado_Acre                                                                                                     236.7471     50.996      4.643      0.000     136.796     336.699
Estado_Alagoas                                                                                                  201.3193     35.327      5.699      0.000     132.078     270.560
Estado_Amapá                                                                                                    258.6712     67.078      3.856      0.000     127.199     390.144
Estado_Amazonas                                                                                                 214.8486     39.222      5.478      0.000     137.974     291.723
Estado_Bahia                                                                                                    180.4888     31.339      5.759      0.000     119.064     241.914
Estado_Ceará                                                                                                    144.2512     31.588      4.567      0.000      82.339     206.164
Estado_Distrito Federal                                                                                          97.8955     41.900      2.336      0.019      15.772     180.019
Estado_Espirito Santo                                                                                           207.5873     32.193      6.448      0.000     144.488     270.687
Estado_Goiás                                                                                                    351.4799     32.163     10.928      0.000     288.441     414.519
Estado_Maranhão                                                                                                 164.4842     28.705      5.730      0.000     108.223     220.746
Estado_Mato Grosso                                                                                              249.4231     35.689      6.989      0.000     179.473     319.373
Estado_Mato Grosso do Sul                                                                                       198.3020     38.907      5.097      0.000     122.043     274.561
Estado_Minas Gerais                                      

In [49]:
x_test8 = x_test7.drop(columns=['Cor_ou_Raça_Indigena'])
regressao_teste8 = regress(y_test, x_test8)
regressao_teste8.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     6109.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:30:41   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57756   BIC:                         1.011e+06
Df Model:                          53                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5332.8165     20.284    262.902      0.000    5293.059    5372.574
Idade                                                                                                             3.3684      0.574      5.870      0.000       2.244       4.493
Estado_Acre                                                                                                     236.8063     50.995      4.644      0.000     136.856     336.757
Estado_Alagoas                                                                                                  201.3029     35.327      5.698      0.000     132.062     270.544
Estado_Amapá                                                                                                    258.6487     67.077      3.856      0.000     127.177     390.120
Estado_Amazonas                                                                                                 215.7389     39.157      5.510      0.000     138.991     292.487
Estado_Bahia                                                                                                    180.3603     31.337      5.755      0.000     118.939     241.781
Estado_Ceará                                                                                                    144.4036     31.585      4.572      0.000      82.496     206.311
Estado_Distrito Federal                                                                                          97.8119     41.899      2.334      0.020      15.690     179.934
Estado_Espirito Santo                                                                                           207.4553     32.192      6.444      0.000     144.360     270.551
Estado_Goiás                                                                                                    351.4402     32.162     10.927      0.000     288.402     414.479
Estado_Maranhão                                                                                                 164.4449     28.704      5.729      0.000     108.184     220.706
Estado_Mato Grosso                                                                                              249.4342     35.688      6.989      0.000     179.485     319.383
Estado_Mato Grosso do Sul                                                                                       198.5276     38.903      5.103      0.000     122.277     274.778
Estado_Minas Gerais                                      

In [52]:
x_test9 = x_test8.drop(columns=['Cor_ou_Raça_Parda'])
regressao_teste9 = regress(y_test, x_test9)
regressao_teste9.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     6227.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:31:36   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57757   BIC:                         1.011e+06
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5341.3283     17.919    298.076      0.000    5306.206    5376.450
Idade                                                                                                             3.3587      0.574      5.854      0.000       2.234       4.483
Estado_Acre                                                                                                     238.1005     50.974      4.671      0.000     138.191     338.010
Estado_Alagoas                                                                                                  202.2806     35.310      5.729      0.000     133.073     271.488
Estado_Amapá                                                                                                    259.6524     67.068      3.872      0.000     128.200     391.105
Estado_Amazonas                                                                                                 217.3424     39.116      5.556      0.000     140.675     294.010
Estado_Bahia                                                                                                    178.3757     31.259      5.706      0.000     117.109     239.643
Estado_Ceará                                                                                                    145.7097     31.552      4.618      0.000      83.869     207.551
Estado_Distrito Federal                                                                                          97.7170     41.899      2.332      0.020      15.595     179.839
Estado_Espirito Santo                                                                                           207.3085     32.191      6.440      0.000     144.214     270.403
Estado_Goiás                                                                                                    352.0588     32.155     10.949      0.000     289.035     415.082
Estado_Maranhão                                                                                                 164.5280     28.704      5.732      0.000     108.268     220.788
Estado_Mato Grosso                                                                                              249.7762     35.686      6.999      0.000     179.831     319.721
Estado_Mato Grosso do Sul                                                                                       198.8341     38.901      5.111      0.000     122.587     275.081
Estado_Minas Gerais                                      

In [54]:
x_test10 = x_test9.drop(columns=['Cor_ou_Raça_Branca'])
regressao_teste10 = regress(y_test, x_test10)
regressao_teste10.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     6349.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:32:21   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57758   BIC:                         1.011e+06
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5345.3405     17.664    302.605      0.000    5310.718    5379.963
Idade                                                                                                             3.4034      0.573      5.942      0.000       2.281       4.526
Estado_Acre                                                                                                     234.8614     50.917      4.613      0.000     135.064     334.658
Estado_Alagoas                                                                                                  201.3005     35.302      5.702      0.000     132.108     270.493
Estado_Amapá                                                                                                    256.7742     67.033      3.831      0.000     125.389     388.160
Estado_Amazonas                                                                                                 213.9997     39.036      5.482      0.000     137.490     290.510
Estado_Bahia                                                                                                    175.8552     31.202      5.636      0.000     114.700     237.010
Estado_Ceará                                                                                                    144.5687     31.540      4.584      0.000      82.750     206.387
Estado_Distrito Federal                                                                                          97.0017     41.896      2.315      0.021      14.886     179.117
Estado_Espirito Santo                                                                                           208.1892     32.184      6.469      0.000     145.107     271.271
Estado_Goiás                                                                                                    352.3543     32.154     10.958      0.000     289.332     415.377
Estado_Maranhão                                                                                                 162.0440     28.644      5.657      0.000     105.902     218.186
Estado_Mato Grosso                                                                                              249.0634     35.682      6.980      0.000     179.126     319.001
Estado_Mato Grosso do Sul                                                                                       199.8527     38.894      5.138      0.000     123.620     276.086
Estado_Minas Gerais                                      

In [55]:
x_test11 = x_test10.drop(columns=['Escolaridade_Doutorado'])
regressao_teste11 = regress(y_test, x_test11)
regressao_teste11.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     6476.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:32:55   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57759   BIC:                         1.011e+06
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5346.7324     17.635    303.196      0.000    5312.169    5381.296
Idade                                                                                                             3.4173      0.573      5.967      0.000       2.295       4.540
Estado_Acre                                                                                                     234.8068     50.917      4.612      0.000     135.009     334.604
Estado_Alagoas                                                                                                  201.1060     35.302      5.697      0.000     131.913     270.299
Estado_Amapá                                                                                                    257.2484     67.033      3.838      0.000     125.864     388.633
Estado_Amazonas                                                                                                 214.0511     39.036      5.483      0.000     137.541     290.562
Estado_Bahia                                                                                                    175.8760     31.202      5.637      0.000     114.720     237.032
Estado_Ceará                                                                                                    144.5678     31.540      4.584      0.000      82.749     206.387
Estado_Distrito Federal                                                                                          97.2242     41.896      2.321      0.020      15.109     179.340
Estado_Espirito Santo                                                                                           208.5698     32.183      6.481      0.000     145.490     271.649
Estado_Goiás                                                                                                    352.4147     32.155     10.960      0.000     289.392     415.438
Estado_Maranhão                                                                                                 161.8868     28.644      5.652      0.000     105.745     218.029
Estado_Mato Grosso                                                                                              248.8109     35.682      6.973      0.000     178.874     318.748
Estado_Mato Grosso do Sul                                                                                       199.8847     38.894      5.139      0.000     123.651     276.118
Estado_Minas Gerais                                      

In [56]:
x_test12 = x_test11.drop(columns=['Escolaridade_Pré-escola'])
regressao_teste12 = regress(y_test, x_test12)
regressao_teste12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     6608.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:33:37   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57760   BIC:                         1.011e+06
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5346.7983     17.635    303.197      0.000    5312.234    5381.363
Idade                                                                                                             3.3772      0.572      5.903      0.000       2.256       4.499
Estado_Acre                                                                                                     235.0848     50.917      4.617      0.000     135.287     334.883
Estado_Alagoas                                                                                                  200.9369     35.303      5.692      0.000     131.744     270.130
Estado_Amapá                                                                                                    257.7962     67.033      3.846      0.000     126.412     389.180
Estado_Amazonas                                                                                                 214.3003     39.036      5.490      0.000     137.790     290.811
Estado_Bahia                                                                                                    175.0750     31.198      5.612      0.000     113.927     236.223
Estado_Ceará                                                                                                    144.5752     31.541      4.584      0.000      82.755     206.395
Estado_Distrito Federal                                                                                          97.6698     41.895      2.331      0.020      15.555     179.784
Estado_Espirito Santo                                                                                           208.7273     32.184      6.486      0.000     145.647     271.807
Estado_Goiás                                                                                                    352.2954     32.155     10.956      0.000     289.272     415.319
Estado_Maranhão                                                                                                 161.1113     28.640      5.625      0.000     104.978     217.245
Estado_Mato Grosso                                                                                              248.5978     35.682      6.967      0.000     178.660     318.535
Estado_Mato Grosso do Sul                                                                                       200.1079     38.895      5.145      0.000     123.874     276.342
Estado_Minas Gerais                                      

In [57]:
x_test13 = x_test12.drop(columns=['Escolaridade_Alfabetização de jovens e adultos'])
regressao_teste13 = regress(y_test, x_test13)
regressao_teste13.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     6745.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:34:22   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57761   BIC:                         1.011e+06
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5347.0230     17.635    303.212      0.000    5312.459    5381.587
Idade                                                                                                             3.3053      0.571      5.792      0.000       2.187       4.424
Estado_Acre                                                                                                     235.3457     50.918      4.622      0.000     135.546     335.145
Estado_Alagoas                                                                                                  200.3390     35.302      5.675      0.000     131.148     269.530
Estado_Amapá                                                                                                    258.7586     67.032      3.860      0.000     127.376     390.141
Estado_Amazonas                                                                                                 214.7889     39.036      5.502      0.000     138.279     291.299
Estado_Bahia                                                                                                    175.1935     31.198      5.615      0.000     114.045     236.342
Estado_Ceará                                                                                                    144.3602     31.541      4.577      0.000      82.540     206.181
Estado_Distrito Federal                                                                                          98.4357     41.894      2.350      0.019      16.324     180.547
Estado_Espirito Santo                                                                                           208.9172     32.184      6.491      0.000     145.836     271.998
Estado_Goiás                                                                                                    352.2257     32.155     10.954      0.000     289.201     415.250
Estado_Maranhão                                                                                                 160.5896     28.639      5.607      0.000     104.458     216.722
Estado_Mato Grosso                                                                                              248.1595     35.682      6.955      0.000     178.222     318.097
Estado_Mato Grosso do Sul                                                                                       200.4969     38.895      5.155      0.000     124.263     276.731
Estado_Minas Gerais                                      

In [58]:
x_test14 = x_test13.drop(columns=['Estado_Roraima'])
regressao_teste13 = regress(y_test, x_test14)
regressao_teste13.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     6745.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:35:11   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57761   BIC:                         1.011e+06
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5406.7688     37.307    144.925      0.000    5333.646    5479.891
Idade                                                                                                             3.3053      0.571      5.792      0.000       2.187       4.424
Estado_Acre                                                                                                     128.3011     79.224      1.619      0.105     -26.978     283.580
Estado_Alagoas                                                                                                   93.2945     69.478      1.343      0.179     -42.882     229.471
Estado_Amapá                                                                                                    151.7141     91.245      1.663      0.096     -27.127     330.555
Estado_Amazonas                                                                                                 107.7444     71.585      1.505      0.132     -32.562     248.051
Estado_Bahia                                                                                                     68.1489     67.354      1.012      0.312     -63.866     200.164
Estado_Ceará                                                                                                     37.3157     67.521      0.553      0.581     -95.026     169.657
Estado_Distrito Federal                                                                                          -8.6088     73.339     -0.117      0.907    -152.353     135.135
Estado_Espirito Santo                                                                                           101.8727     67.876      1.501      0.133     -31.164     234.910
Estado_Goiás                                                                                                    245.1812     67.866      3.613      0.000     112.163     378.200
Estado_Maranhão                                                                                                  53.5451     66.095      0.810      0.418     -76.001     183.092
Estado_Mato Grosso                                                                                              141.1150     69.725      2.024      0.043       4.454     277.776
Estado_Mato Grosso do Sul                                                                                        93.4523     71.565      1.306      0.192     -46.815     233.720
Estado_Minas Gerais                                      

In [60]:
x_test15 = x_test14.drop(columns=['Estado_Distrito Federal'])
regressao_teste15 = regress(y_test, x_test15)
regressao_teste15.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     6889.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:36:39   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57762   BIC:                         1.011e+06
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5403.5597     25.386    212.855      0.000    5353.803    5453.317
Idade                                                                                                             3.3050      0.571      5.791      0.000       2.186       4.424
Estado_Acre                                                                                                     133.9699     62.803      2.133      0.033      10.876     257.064
Estado_Alagoas                                                                                                   98.9535     50.030      1.978      0.048       0.895     197.012
Estado_Amapá                                                                                                    157.3848     77.407      2.033      0.042       5.666     309.103
Estado_Amazonas                                                                                                 113.4028     52.922      2.143      0.032       9.676     217.130
Estado_Bahia                                                                                                     73.8047     47.066      1.568      0.117     -18.444     166.054
Estado_Ceará                                                                                                     42.9737     47.284      0.909      0.363     -49.704     135.651
Estado_Espirito Santo                                                                                           107.5620     47.518      2.264      0.024      14.426     200.698
Estado_Goiás                                                                                                    250.8736     47.479      5.284      0.000     157.814     343.933
Estado_Maranhão                                                                                                  59.1929     45.317      1.306      0.191     -29.628     148.014
Estado_Mato Grosso                                                                                              146.8095     50.081      2.931      0.003      48.651     244.968
Estado_Mato Grosso do Sul                                                                                        99.1482     52.602      1.885      0.059      -3.953     202.249
Estado_Minas Gerais                                                                                             161.5586     40.962      3.944      0.000      81.274     241.844
Estado_Paraná                                            

In [61]:
x_test16 = x_test15.drop(columns=['Estado_Piaui'])
regressao_teste16 = regress(y_test, x_test16)
regressao_teste16.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     7038.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:37:21   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57763   BIC:                         1.011e+06
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5410.1710     22.792    237.368      0.000    5365.498    5454.844
Idade                                                                                                             3.3083      0.571      5.797      0.000       2.190       4.427
Estado_Acre                                                                                                     121.4301     59.115      2.054      0.040       5.564     237.297
Estado_Alagoas                                                                                                   86.2514     45.185      1.909      0.056      -2.311     174.814
Estado_Amapá                                                                                                    144.8155     74.432      1.946      0.052      -1.072     290.703
Estado_Amazonas                                                                                                 100.7214     48.383      2.082      0.037       5.890     195.552
Estado_Bahia                                                                                                     61.0191     41.807      1.460      0.144     -20.923     142.961
Estado_Ceará                                                                                                     30.2043     42.067      0.718      0.473     -52.247     112.655
Estado_Espirito Santo                                                                                            95.1533     42.635      2.232      0.026      11.588     178.718
Estado_Goiás                                                                                                    238.4811     42.605      5.597      0.000     154.975     321.987
Estado_Maranhão                                                                                                  46.3600     39.784      1.165      0.244     -31.617     124.337
Estado_Mato Grosso                                                                                              134.4688     45.527      2.954      0.003      45.236     223.701
Estado_Mato Grosso do Sul                                                                                        86.7836     48.269      1.798      0.072      -7.824     181.392
Estado_Minas Gerais                                                                                             149.0819     35.111      4.246      0.000      80.263     217.900
Estado_Paraná                                            

In [62]:
x_test17 = x_test16.drop(columns=['Estado_Pará'])
regressao_teste17 = regress(y_test, x_test17)
regressao_teste17.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     7195.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:38:12   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57764   BIC:                         1.011e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5413.9745     20.894    259.118      0.000    5373.022    5454.927
Idade                                                                                                             3.3073      0.571      5.796      0.000       2.189       4.426
Estado_Acre                                                                                                     114.2461     56.557      2.020      0.043       3.393     225.099
Estado_Alagoas                                                                                                   79.0143     41.730      1.893      0.058      -2.777     160.806
Estado_Amapá                                                                                                    137.6391     72.421      1.901      0.057      -4.307     279.585
Estado_Amazonas                                                                                                  93.4919     45.180      2.069      0.039       4.938     182.046
Estado_Bahia                                                                                                     53.7487     38.010      1.414      0.157     -20.752     128.249
Estado_Ceará                                                                                                     22.9501     38.314      0.599      0.549     -52.145      98.045
Estado_Espirito Santo                                                                                            88.0030     39.047      2.254      0.024      11.471     164.535
Estado_Goiás                                                                                                    231.3333     39.017      5.929      0.000     154.861     307.806
Estado_Maranhão                                                                                                  39.0709     35.752      1.093      0.274     -31.003     109.145
Estado_Mato Grosso                                                                                              127.3371     42.203      3.017      0.003      44.619     210.055
Estado_Mato Grosso do Sul                                                                                        79.6543     45.150      1.764      0.078      -8.840     168.149
Estado_Minas Gerais                                                                                             141.9075     30.622      4.634      0.000      81.888     201.927
Estado_Paraná                                            

In [63]:
x_test18 = x_test17.drop(columns=['Estado_Sergipe'])
regressao_teste18 = regress(y_test, x_test18)
regressao_teste18.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     7358.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:38:59   Log-Likelihood:            -5.0539e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57765   BIC:                         1.011e+06
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5416.4175     20.406    265.429      0.000    5376.421    5456.414
Idade                                                                                                             3.3084      0.571      5.798      0.000       2.190       4.427
Estado_Acre                                                                                                     109.5554     55.897      1.960      0.050      -0.003     219.114
Estado_Alagoas                                                                                                   74.2167     40.789      1.820      0.069      -5.730     154.163
Estado_Amapá                                                                                                    132.9329     71.903      1.849      0.064      -7.998     273.863
Estado_Amazonas                                                                                                  88.7399     44.329      2.002      0.045       1.855     175.625
Estado_Bahia                                                                                                     48.9108     36.957      1.323      0.186     -23.525     121.346
Estado_Ceará                                                                                                     18.1218     37.273      0.486      0.627     -54.933      91.177
Estado_Espirito Santo                                                                                            83.3541     38.102      2.188      0.029       8.675     158.033
Estado_Goiás                                                                                                    226.6854     38.071      5.954      0.000     152.066     301.305
Estado_Maranhão                                                                                                  34.2171     34.622      0.988      0.323     -33.643     102.077
Estado_Mato Grosso                                                                                              122.7244     41.344      2.968      0.003      41.691     203.758
Estado_Mato Grosso do Sul                                                                                        75.0313     44.344      1.692      0.091     -11.884     161.946
Estado_Minas Gerais                                                                                             137.2245     29.389      4.669      0.000      79.621     194.828
Estado_Paraná                                            

In [64]:
x_test19 = x_test18.drop(columns=['Estado_Ceará'])
regressao_teste19 = regress(y_test, x_test19)
regressao_teste19.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     7530.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:39:58   Log-Likelihood:            -5.0540e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57766   BIC:                         1.011e+06
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5419.0218     19.691    275.209      0.000    5380.428    5457.615
Idade                                                                                                             3.3102      0.571      5.801      0.000       2.192       4.429
Estado_Acre                                                                                                     104.5588     54.944      1.903      0.057      -3.131     212.249
Estado_Alagoas                                                                                                   69.1307     39.424      1.753      0.080      -8.141     146.403
Estado_Amapá                                                                                                    127.9127     71.157      1.798      0.072     -11.556     267.382
Estado_Amazonas                                                                                                  83.6511     43.075      1.942      0.052      -0.777     168.079
Estado_Bahia                                                                                                     43.7633     35.408      1.236      0.216     -25.636     113.162
Estado_Espirito Santo                                                                                            78.4130     36.721      2.135      0.033       6.440     150.386
Estado_Goiás                                                                                                    221.7526     36.694      6.043      0.000     149.832     293.673
Estado_Maranhão                                                                                                  29.0386     32.943      0.881      0.378     -35.530      93.607
Estado_Mato Grosso                                                                                              117.8131     40.090      2.939      0.003      39.236     196.390
Estado_Mato Grosso do Sul                                                                                        70.1242     43.180      1.624      0.104     -14.509     154.757
Estado_Minas Gerais                                                                                             132.2550     27.555      4.800      0.000      78.248     186.262
Estado_Paraná                                                                                                    92.6535     31.155      2.974      0.003      31.590     153.717
Estado_Paraíba                                           

In [68]:
x_test20 = x_test19.drop(columns=['Estado_Rio Grande do Norte'])
regressao_teste20 = regress(y_test, x_test20)
regressao_teste20.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     7709.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:43:15   Log-Likelihood:            -5.0540e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57767   BIC:                         1.011e+06
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5420.5190     19.497    278.018      0.000    5382.305    5458.733
Idade                                                                                                             3.3134      0.571      5.807      0.000       2.195       4.432
Estado_Acre                                                                                                     101.5966     54.673      1.858      0.063      -5.562     208.755
Estado_Alagoas                                                                                                   66.1587     39.043      1.694      0.090     -10.367     142.684
Estado_Amapá                                                                                                    124.9520     70.948      1.761      0.078     -14.107     264.011
Estado_Amazonas                                                                                                  80.6902     42.730      1.888      0.059      -3.060     164.441
Estado_Bahia                                                                                                     40.7902     34.983      1.166      0.244     -27.776     109.356
Estado_Espirito Santo                                                                                            75.4343     36.310      2.078      0.038       4.267     146.602
Estado_Goiás                                                                                                    218.7605     36.279      6.030      0.000     147.654     289.867
Estado_Maranhão                                                                                                  26.0815     32.491      0.803      0.422     -37.601      89.764
Estado_Mato Grosso                                                                                              114.8396     39.715      2.892      0.004      36.997     192.682
Estado_Mato Grosso do Sul                                                                                        67.1582     42.834      1.568      0.117     -16.796     151.113
Estado_Minas Gerais                                                                                             129.2740     27.003      4.787      0.000      76.347     182.201
Estado_Paraná                                                                                                    89.6781     30.670      2.924      0.003      29.564     149.792
Estado_Paraíba                                           

In [69]:
x_test21 = x_test20.drop(columns=['Estado_Rondônia'])
regressao_teste21 = regress(y_test, x_test21)
regressao_teste21.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Rendimento_Bruto   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     7897.
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:43:16   Log-Likelihood:            -5.0540e+05
No. Observations:               57810   AIC:                         1.011e+06
Df Residuals:                   57768   BIC:                         1.011e+06
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================================
                                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                                          5422.1668     19.292    281.059      0.000    5384.355    5459.979
Idade                                                                                                             3.3097      0.571      5.801      0.000       2.191       4.428
Estado_Acre                                                                                                      98.5407     54.422      1.811      0.070      -8.126     205.208
Estado_Alagoas                                                                                                   63.2580     38.726      1.633      0.102     -12.646     139.162
Estado_Amapá                                                                                                    121.9607     70.763      1.724      0.085     -16.735     260.657
Estado_Amazonas                                                                                                  77.7375     42.430      1.832      0.067      -5.425     160.900
Estado_Bahia                                                                                                     37.9534     34.644      1.096      0.273     -29.949     105.856
Estado_Espirito Santo                                                                                            72.3257     35.918      2.014      0.044       1.927     142.725
Estado_Goiás                                                                                                    215.6674     35.890      6.009      0.000     145.322     286.013
Estado_Maranhão                                                                                                  23.2420     32.125      0.723      0.469     -39.724      86.208
Estado_Mato Grosso                                                                                              111.7046     39.351      2.839      0.005      34.576     188.833
Estado_Mato Grosso do Sul                                                                                        64.0278     42.497      1.507      0.132     -19.267     147.323
Estado_Minas Gerais                                                                                             126.2196     26.493      4.764      0.000      74.294     178.145
Estado_Paraná                                                                                                    86.5275     30.193      2.866      0.004      27.350     145.705
Estado_Paraíba                                           

In [71]:
x_test22, perdeu = remove_maior(regressao_teste21, x_test21)

AttributeError: 'Summary' object has no attribute 'pvalues'